# 10 Extract 2025 Qualifying Results

Get actual qualifying results for validation.

**For normal weekends:** Qualifying results
**For sprint weekends:** Sprint Qualifying results (what we predict)

In [1]:
import fastf1 as ff1
import pandas as pd
import json
from pathlib import Path

import logging
logging.getLogger("fastf1").setLevel(logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

ff1.Cache.enable_cache('../data/raw/.fastf1_cache')

season = 2025

## Extract All Qualifying Results

In [2]:
calendar = ff1.get_event_schedule(season)

print(f"Extracting qualifying results: {season}")
print("=" * 70)

results = {}

for idx, event in calendar.iterrows():
    event_name = event['EventName']
    
    if 'Testing' in str(event_name) or pd.isna(event_name):
        continue
    
    print(f"\n{event_name}")
    
    # Check weekend type
    event_format = str(event.get('EventFormat', '')).lower()
    is_sprint = 'sprint' in event_format
    
    # For sprint: use Sprint Qualifying (what we predict)
    # For normal: use Qualifying
    session_name = 'Sprint Qualifying' if is_sprint else 'Qualifying'
    
    try:
        session = ff1.get_session(season, event_name, session_name)
        session.load(laps=False, telemetry=False, weather=False)
        
        session_results = []
        
        # Try results dataframe first
        if hasattr(session, 'results') and session.results is not None and len(session.results) > 0:
            for idx, row in session.results.iterrows():
                if pd.notna(row.get('Position')):
                    session_results.append({
                        'position': int(row['Position']),
                        'driver': row['Abbreviation'],
                        'team': row['TeamName'],
                        'time': str(row.get('Q3', row.get('Q2', row.get('Q1', 'N/A'))))
                    })
        
        # Sprint Qualifying sometimes doesn't populate results - use laps
        if not session_results:
            session.load(laps=True, telemetry=False, weather=False)
            
            if hasattr(session, 'laps') and session.laps is not None and len(session.laps) > 0:
                # Get fastest lap per driver
                fastest = session.laps.groupby(['Driver', 'Team'])['LapTime'].min().reset_index()
                fastest = fastest.sort_values('LapTime')
                
                for pos, (_, row) in enumerate(fastest.iterrows(), 1):
                    session_results.append({
                        'position': pos,
                        'driver': row['Driver'],
                        'team': row['Team'],
                        'time': str(row['LapTime'])
                    })
        
        if session_results:
            results[event_name] = {
                'weekend_type': 'sprint' if is_sprint else 'normal',
                'session': session_name,
                'positions': session_results
            }
            
            print(f"  🟢 {session_name}: {len(session_results)} positions")
        else:
            print(f"  🔴 No results available")
            
    except Exception as e:
        print(f"  🔴 Failed: {e}")

print(f"\n" + "=" * 70)
print(f"Extracted {len(results)} races")

Extracting qualifying results: 2025

Australian Grand Prix
  🟢 Qualifying: 20 positions

Chinese Grand Prix
  🟢 Sprint Qualifying: 20 positions

Japanese Grand Prix
  🟢 Qualifying: 20 positions

Bahrain Grand Prix
  🟢 Qualifying: 20 positions

Saudi Arabian Grand Prix
  🟢 Qualifying: 20 positions

Miami Grand Prix
  🟢 Sprint Qualifying: 20 positions

Emilia Romagna Grand Prix
  🟢 Qualifying: 20 positions

Monaco Grand Prix
  🟢 Qualifying: 20 positions

Spanish Grand Prix
  🟢 Qualifying: 20 positions

Canadian Grand Prix
  🟢 Qualifying: 20 positions

Austrian Grand Prix
  🟢 Qualifying: 20 positions

British Grand Prix
  🟢 Qualifying: 20 positions

Belgian Grand Prix
  🟢 Sprint Qualifying: 20 positions

Hungarian Grand Prix
  🟢 Qualifying: 20 positions

Dutch Grand Prix
  🟢 Qualifying: 20 positions

Italian Grand Prix
  🟢 Qualifying: 20 positions

Azerbaijan Grand Prix
  🟢 Qualifying: 20 positions

Singapore Grand Prix
  🟢 Qualifying: 20 positions

United States Grand Prix
  🟢 Sprint Qua

## Quick Check

In [3]:
# Check first race
if results:
    first_race = list(results.keys())[0]
    first_result = results[first_race]
    
    print(f"{first_race} ({first_result['weekend_type']} weekend)")
    print("=" * 70)
    print(f"Session: {first_result['session']}")
    print(f"\nTop 10:")
    
    for pos in first_result['positions'][:10]:
        print(f"  {pos['position']}. {pos['team']:<25} {pos['driver']}")
else:
    print("No results extracted!")

Australian Grand Prix (normal weekend)
Session: Qualifying

Top 10:
  1. McLaren                   NOR
  2. McLaren                   PIA
  3. Red Bull Racing           VER
  4. Mercedes                  RUS
  5. Racing Bulls              TSU
  6. Williams                  ALB
  7. Ferrari                   LEC
  8. Ferrari                   HAM
  9. Alpine                    GAS
  10. Williams                  SAI


## Save Results

In [4]:
output = {
    'season': season,
    'total_races': len(results),
    'races': results
}

output_path = Path(f'../data/processed/testing_files/validation/2025_qualifying_results.json')
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, 'w') as f:
    json.dump(output, f, indent=2)

print(f"🟢 Saved to {output_path}")
print(f"  Races: {len(results)}")
print(f"  File size: {output_path.stat().st_size / 1024:.1f} KB")

🟢 Saved to ../data/processed/testing_files/validation/2025_qualifying_results.json
  Races: 24
  File size: 69.2 KB
